In [1]:
from importlib.metadata import version
from torch.utils.data import Dataset,DataLoader
import re,torch,tiktoken
print(f"PyTorch version: {version('torch')}\nTiktoken version: {version('tiktoken')}")

PyTorch version: 2.9.1
Tiktoken version: 0.12.0


# Tokenizing Text

In [2]:
with open('./1-adding_bells_whistles_to_training_loop/verdict.txt','r',encoding='utf-8') as f:
    raw_text=f.read()
print(f'Total number of characters: {len(raw_text)}\n{raw_text[:99]}')

Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [3]:
text='''Hello, world.
This's a test.'''
result=re.split(r'(\s)',text)
result

['Hello,', ' ', 'world.', '\n', "This's", ' ', 'a', ' ', 'test.']

In [4]:
result=re.split(r'([,.]|\s)',text)
result

['Hello',
 ',',
 '',
 ' ',
 'world',
 '.',
 '',
 '\n',
 "This's",
 ' ',
 'a',
 ' ',
 'test',
 '.',
 '']

In [5]:
result=[item for item in result if item.strip()]
result

['Hello', ',', 'world', '.', "This's", 'a', 'test', '.']

In [6]:
text='''Hello, world.
Is this a test?'''
result=re.split(r'([,.:;?_!"()\']|\s)',text)
result=[item.strip() for item in result if item.strip()]
result

['Hello', ',', 'world', '.', 'Is', 'this', 'a', 'test', '?']

In [7]:
preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)
preprocessed=[item.strip() for item in preprocessed if item.strip()]
preprocessed[:30]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius',
 '--',
 'though',
 'a',
 'good',
 'fellow',
 'enough',
 '--',
 'so',
 'it',
 'was',
 'no',
 'great',
 'surprise',
 'to',
 'me',
 'to',
 'hear',
 'that',
 ',',
 'in']

In [8]:
len(preprocessed)

4690

# Converting Tokens Into Token IDs

In [9]:
all_words=sorted(set(preprocessed))
vocab_size=len(all_words)
vocab_size

1130

In [10]:
vocab={token:integer for integer,token in enumerate(all_words)}
for i,item in enumerate(vocab.items()):
    print(item)
    if i>=50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [11]:
class SimpleTokenizerV1:
    def __init__(self,vocab):
        self.str_to_int=vocab
        self.int_to_str={i:s for s,i in vocab.items()}
    def encode(self,text):
        preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed=[item.strip() for item in preprocessed if item.strip()]
        ids=[self.str_to_int[s] for s in preprocessed]
        return ids
    def decode(self,ids):
        text=' '.join([self.int_to_str[i] for i in ids])
        text=re.sub(r'\s+([,.?!"()\'])',r'\1',text)
        return text
tokenizer=SimpleTokenizerV1(vocab)
text='''"It's the last he painted, you know."
Mrs. Gisburn said with pardonable pride.'''
ids=tokenizer.encode(text)
ids

[1,
 56,
 2,
 850,
 988,
 602,
 533,
 746,
 5,
 1126,
 596,
 7,
 1,
 67,
 7,
 38,
 851,
 1108,
 754,
 793,
 7]

In [12]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know." Mrs. Gisburn said with pardonable pride.'

# Adding Special Context Tokens

In [13]:
all_tokens=sorted(list(set(preprocessed)))
all_tokens.extend(['<|endoftext|>','<|unk|>'])
vocab={token:integer for integer,token in enumerate(all_tokens)}
len(vocab.items())

1132

In [14]:
for i,item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [15]:
class SimpleTokenizerV2:
    def __init__(self,vocab):
        self.str_to_int=vocab
        self.int_to_str={i:s for s,i in vocab.items()}
    def encode(self,text):
        preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed=[item.strip() for item in preprocessed if item.strip()]
        preprocessed=[item if item in self.str_to_int else '<|unk|>' for item in preprocessed]
        ids=[self.str_to_int[s] for s in preprocessed]
        return ids
    def decode(self,ids):
        text=' '.join([self.int_to_str[i] for i in ids])
        text=re.sub(r'\s+([,.:;?!"()\'])',r'\1',text)
        return text
tokenizer=SimpleTokenizerV2(vocab)
text1='Hello, do you like tea?'
text2='In the sunlit terraces of the palace.'
text=' <|endoftext|> '.join((text1,text2))
text

'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.'

In [16]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [17]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

# BPE

In [18]:
tokenizer=tiktoken.get_encoding('gpt2')
text=('Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.')
integers=tokenizer.encode(text,allowed_special={'<|endoftext|>'})
integers

[15496,
 11,
 466,
 345,
 588,
 8887,
 30,
 220,
 50256,
 554,
 262,
 4252,
 18250,
 8812,
 2114,
 286,
 617,
 34680,
 27271,
 13]

In [19]:
strings=tokenizer.decode(integers)
strings

'Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.'

# Data Sampling With Sliding Window

In [20]:
enc_text=tokenizer.encode(raw_text)
len(enc_text)

5145

In [21]:
enc_sample=enc_text[50:]
context_size=4
x=enc_sample[:context_size]
y=enc_sample[1:context_size+1]
print(f'x: {x}\ny: {y}')

x: [290, 4920, 2241, 287]
y: [4920, 2241, 287, 257]


In [22]:
for i in range(1,context_size+1):
    context=enc_sample[:i]
    desired=enc_sample[i]
    print(f'{context} -> {desired}')

[290] -> 4920
[290, 4920] -> 2241
[290, 4920, 2241] -> 287
[290, 4920, 2241, 287] -> 257


In [23]:
for i in range(1,context_size+1):
    context=enc_sample[:i]
    desired=enc_sample[i]
    print(f'{tokenizer.decode(context)} -> {tokenizer.decode([desired])}')

 and ->  established
 and established ->  himself
 and established himself ->  in
 and established himself in ->  a


In [24]:
class GPTDatasetV1(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        self.input_ids=[]
        self.target_ids=[]
        token_ids=tokenizer.encode(txt,allowed_special={'<|endoftext|>'})
        assert len(token_ids)>max_length,'Number of tokenized inputs must at least be =max_length+1.'
        for i in range(0,len(token_ids)-max_length,stride):
            input_chunk=token_ids[i:i+max_length]
            target_chunk=token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self,idx):
        return self.input_ids[idx],self.target_ids[idx]
def create_dataloader_v1(txt,batch_size=4,max_length=256,stride=128,shuffle=True,drop_last=True,num_workers=0):
    tokenizer=tiktoken.get_encoding('gpt2')
    dataset=GPTDatasetV1(txt,tokenizer,max_length,stride)
    dataloader=DataLoader(dataset,batch_size=batch_size,shuffle=shuffle,drop_last=drop_last,num_workers=num_workers)
    return dataloader
dataloader=create_dataloader_v1(raw_text,batch_size=1,max_length=4,stride=1,shuffle=False)
data_iter=iter(dataloader)
first_batch=next(data_iter)
first_batch

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]

In [25]:
second_batch=next(data_iter)
second_batch

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]

In [26]:
dataloader=create_dataloader_v1(raw_text,batch_size=8,max_length=4,stride=4,shuffle=False)
data_iter=iter(dataloader)
inputs,targets=next(data_iter)
print(f'Inputs:\n{inputs}\nTargets:\n{targets}')

Inputs:
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Targets:
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


# Creating Token Embeddings

In [27]:
input_ids=torch.tensor([2,3,5,1])
vocab_size=6
output_dim=3
embedding_layer=torch.nn.Embedding(vocab_size,output_dim)
embedding_layer.weight

Parameter containing:
tensor([[ 0.1545,  1.1995, -1.6787],
        [-1.8155, -0.5289,  0.6125],
        [-1.0188,  0.8816, -0.8616],
        [ 0.0559, -1.9115,  2.3488],
        [ 0.5575, -1.1588,  0.3312],
        [ 1.0654,  0.1901, -0.2692]], requires_grad=True)

In [28]:
embedding_layer(torch.tensor([3]))

tensor([[ 0.0559, -1.9115,  2.3488]], grad_fn=<EmbeddingBackward0>)

In [29]:
embedding_layer(input_ids)

tensor([[-1.0188,  0.8816, -0.8616],
        [ 0.0559, -1.9115,  2.3488],
        [ 1.0654,  0.1901, -0.2692],
        [-1.8155, -0.5289,  0.6125]], grad_fn=<EmbeddingBackward0>)

# Encoding Word Positions

In [30]:
vocab_size=50257
output_dim=256
token_embedding_layer=torch.nn.Embedding(vocab_size,output_dim)
max_length=4
dataloader=create_dataloader_v1(raw_text,batch_size=8,max_length=max_length,stride=max_length,shuffle=False)
data_iter=iter(dataloader)
inputs,targets=next(data_iter)
print(f'Token IDs:\n{inputs}\nInputs shape: {inputs.shape}')

Token IDs:
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Inputs shape: torch.Size([8, 4])


In [31]:
token_embeddings=token_embedding_layer(inputs)
token_embeddings.shape

torch.Size([8, 4, 256])

In [32]:
context_length=max_length
pos_embedding_layer=torch.nn.Embedding(context_length,output_dim)
pos_embeddings=pos_embedding_layer(torch.arange(max_length))
pos_embeddings.shape

torch.Size([4, 256])